### Introduction

In this notebook, I use genius API and lyricgenius library to fetch lyrics for the songs with missing lyrics in the dataset.

In [14]:
import lyricsgenius
import pandas as pd

In [15]:
from requests.exceptions import Timeout

In [16]:
# This is run only the first time

# set up Genius API
genius = lyricsgenius.Genius('thiswasmysecretapi', timeout=10, sleep_time=5)

# load the dataset
df_fin = pd.read_csv('datasets/all_songs_with_some_lyrics.csv')

In [64]:
# this is for later iterations to continue after an error

# set up Genius API
genius = lyricsgenius.Genius('thiswasmysecretapi', timeout=10, sleep_time=5)

# reload the saved dataset
df_fin = pd.read_csv('datasets/all_songs_with_more_genius_lyrics.csv')

In [50]:
df_fin.tail()

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,track_lyrics,artist_gender
28342,7bxnKAamR3snQ1VGLuVfC1,City Of Lights - Official Radio Edit,Lush & Simon,42,2azRoBBWEEEYhqV6sb7JrT,City Of Lights (Vocal Mix),2014-04-28,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,...,1,0.0936,0.076600,0.000000,0.0668,0.2100,128.170,204375,error,Unknown
28343,5Aevni09Em4575077nkWHz,Closer - Sultan & Ned Shepard Remix,Tegan and Sara,20,6kD6KLxj7s8eCE3ABvAyf5,Closer Remixed,2013-03-08,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,...,1,0.0420,0.001710,0.004270,0.3750,0.4000,128.041,353120,error,Female
28344,7ImMqPP3Q1yfUHvsdn7wEo,Sweet Surrender - Radio Edit,Starkillers,14,0ltWNSY9JgxoIZO4VzuCa6,Sweet Surrender (Radio Edit),2014-04-21,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,...,0,0.0481,0.108000,0.000001,0.1500,0.4360,127.989,210112,error,Unknown
28345,2m69mhnfQ1Oq6lGtXuYhgX,Only For You - Maor Levi Remix,Mat Zo,15,1fGrOkHnHJcStl14zNx8Jy,Only For You (Remixes),2014-01-01,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,...,1,0.1090,0.007920,0.127000,0.3430,0.3080,128.008,367432,error,Female
28346,29zWqhca3zt5NsckZqDf6c,Typhoon - Original Mix,Julian Calor,27,0X3mUOm6MhxR7PzxG95rAo,Typhoon/Storm,2014-03-03,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,...,0,0.0385,0.000133,0.341000,0.7420,0.0894,127.984,337500,error,Male


In [65]:
# iterate through the dataset and fetch lyrics for each song
genius.verbose = True
genius.remove_section_headers = True
genius.skip_non_songs = True

for index, row in df_fin.iterrows():
    if pd.isna(row['track_lyrics']) or row['track_lyrics'] == "error":  # check if lyrics are missing
#         if row['track_lyrics'] == "error":
#             print("Re-querying error row...")
        
        artist = row['track_artist']
        song_title = row['track_name']
        
        try:
            # search for the song lyrics
            song = genius.search_song(song_title, artist)

            # store the lyrics in the dataset
            if song is not None:
                print(f"Token count: {len(song.lyrics.split())}")            
                print("Adding lyrics to row...")
                df_fin.at[index, 'track_lyrics'] = song.lyrics
            else:
                # if no results found, assign 'no result' to track_lyrics
                print("No result: Adding 'no result' to row...")
                df_fin.at[index, 'track_lyrics'] = 'no result'

            print(f"Index: {index}\n")

        except:
            print(f"Error occurred for {artist} - {song_title}")
            
            # fill in 'error' for track_lyrics for the song with error
            df_fin.at[index, 'track_lyrics'] = 'error'



Searching for "Nervous - The Ooh Song: Mark McCabe Remix" by Gavin James...
No results found for: 'Nervous - The Ooh Song: Mark McCabe Remix Gavin James'
No result: Adding 'no result' to row...
Index: 15638

Searching for "Don't Forget" by Nephaim...
Done.
Token count: 2718
Adding lyrics to row...
Index: 15732

Searching for "Lovin’" by Hansie...
Done.
Token count: 274
Adding lyrics to row...
Index: 18525

Searching for "Cuba Isla Bella (feat. Gente de Zona, Leoni Torres, Isaac Delgado, Buena Fe, Descemer Bueno, Laritza Bacallao, Waldo Mendoza & Pedrito Martinez)" by Orishas...
Done.
Token count: 490
Adding lyrics to row...
Index: 20031

Searching for "Bailando - Official Salsa Version" by Issac Delgado...
Done.
Token count: 395
Adding lyrics to row...
Index: 20032

Searching for "Ser De Sol" by Descemer Bueno...
Done.
Token count: 168
Adding lyrics to row...
Index: 20033

Searching for "Es Tu Mirada - Remasterizado" by Leoni Torres...
Done.
Token count: 5065
Adding lyrics to row...
In

Done.
Token count: 98383
Adding lyrics to row...
Index: 20094

Searching for "Easy (LIke Sunday Morning)" by Charles Givings...
Done.
Token count: 3642
Adding lyrics to row...
Index: 20095

Searching for "Tell It Like It Is" by Aaron Neville...
Done.
Token count: 170
Adding lyrics to row...
Index: 20096

Searching for "Lust For Life" by Iggy Pop...
Done.
Token count: 385
Adding lyrics to row...
Index: 20099

Searching for "You Dropped A Bomb On Me" by The Gap Band...
Done.
Token count: 647
Adding lyrics to row...
Index: 20100

Searching for "Let's Dance - 2002 Remaster" by David Bowie...
Done.
Token count: 2023
Adding lyrics to row...
Index: 20102

Searching for "It's Not Unusual" by Tom Jones...
Done.
Token count: 156
Adding lyrics to row...
Index: 20103

Searching for "Shake Your Groove Thing" by Peaches & Herb...
Done.
Token count: 361
Adding lyrics to row...
Index: 20104

Searching for "Cissy Strut" by The Meters...
Specified song does not contain lyrics. Rejecting.
No result: Addi

Done.
Token count: 353
Adding lyrics to row...
Index: 20176

Searching for "Foda Que Ela É Linda" by 3030...
Done.
Token count: 374
Adding lyrics to row...
Index: 20177

Searching for "Falando Dela" by Mc Maha...
Done.
Token count: 233
Adding lyrics to row...
Index: 20178

Searching for "Tudo o Que Você Quiser" by Orgânico...
Done.
Token count: 826
Adding lyrics to row...
Index: 20179

Searching for "Se Essa Mina" by San Joe...
Done.
Token count: 838
Adding lyrics to row...
Index: 20180

Searching for "Meu Bem" by Matheus Queiroz...
Done.
Token count: 359
Adding lyrics to row...
Index: 20181

Searching for "Louca Vida" by Crod...
Done.
Token count: 390
Adding lyrics to row...
Index: 20182

Searching for "Fogo" by Lourena...
Done.
Token count: 316
Adding lyrics to row...
Index: 20183

Searching for "Só Nós Dois (Rasta Beats Jam III)" by 1Kilo...
No results found for: 'Só Nós Dois (Rasta Beats Jam III) 1Kilo'
No result: Adding 'no result' to row...
Index: 20184

Searching for "Ela Me Faz

Done.
Token count: 236
Adding lyrics to row...
Index: 20251

Searching for "Lately" by Celeste...
Done.
Token count: 276
Adding lyrics to row...
Index: 20252

Searching for "Do You Love Her Now" by Jai Paul...
Done.
Token count: 284
Adding lyrics to row...
Index: 20254

Searching for "Cellophane" by FKA twigs...
Done.
Token count: 237
Adding lyrics to row...
Index: 20255

Searching for "Hey, Ma" by Bon Iver...
Done.
Token count: 275
Adding lyrics to row...
Index: 20256

Searching for "Free The Frail" by JPEGMAFIA...
Done.
Token count: 513
Adding lyrics to row...
Index: 20257

Searching for "Sunflower (feat. Steve Lacy)" by Vampire Weekend...
Done.
Token count: 505
Adding lyrics to row...
Index: 20258

Searching for "New Love Cassette" by Angel Olsen...
Done.
Token count: 104
Adding lyrics to row...
Index: 20259

Searching for "In My Room" by Frank Ocean...
Done.
Token count: 322
Adding lyrics to row...
Index: 20260

Searching for "Home" by Caribou...
Done.
Token count: 173
Adding lyric

Done.
Token count: 782
Adding lyrics to row...
Index: 20333

Searching for "Slow Down" by Cam Meekins...
Done.
Token count: 812
Adding lyrics to row...
Index: 20334

Searching for "Floats My Boat" by Aer...
Done.
Token count: 559
Adding lyrics to row...
Index: 20335

Searching for "Different Day" by CloutBag...
Done.
Token count: 672
Adding lyrics to row...
Index: 20336

Searching for "On God" by CloutBag...
Done.
Token count: 276
Adding lyrics to row...
Index: 20337

Searching for "My Heart" by Coopex...
Done.
Token count: 362
Adding lyrics to row...
Index: 20338

Searching for "About You" by CCIITTYY...
No results found for: 'About You CCIITTYY'
No result: Adding 'no result' to row...
Index: 20339

Searching for "Like A Fool (Back To You)" by Mabryx...
Done.
Token count: 575
Adding lyrics to row...
Index: 20340

Searching for "Secrets" by Fenris...
Done.
Token count: 267
Adding lyrics to row...
Index: 20341

Searching for "Now That You're Gone" by Avazesa...
Done.
Token count: 728
Ad

...

Done.
Token count: 44
Adding lyrics to row...
Index: 28079

Searching for "The Kick" by Maurice West...
Done.
Token count: 51
Adding lyrics to row...
Index: 28080

Searching for "Seventeen - Thomas Gold Festival Mix" by Thomas Gold...
Done.
Token count: 3972
Adding lyrics to row...
Index: 28081

Searching for "Rave Machine" by METAFO4R...
Done.
Token count: 4380
Adding lyrics to row...
Index: 28082

Searching for "Bounce (feat. Snoop Dogg)" by Dimitri Vegas & Like Mike...
Done.
Token count: 190
Adding lyrics to row...
Index: 28083

Searching for "Wild Wild Son - Club Mix" by Armin van Buuren...
Done.
Token count: 124
Adding lyrics to row...
Index: 28084

Searching for "Pjanoo - Radio Edit" by Eric Prydz...
Done.
Token count: 4059
Adding lyrics to row...
Index: 28085

Searching for "Paradise - Deniz Koyu Festival Mix" by Nicky Romero...
No results found for: 'Paradise - Deniz Koyu Festival Mix Nicky Romero'
No result: Adding 'no result' to row...
Index: 28086

Searching for "Sex, Love &

Done.
Token count: 6731
Adding lyrics to row...
Index: 28141

Searching for "Repeat - Original Mix" by Regal...
Done.
Token count: 508
Adding lyrics to row...
Index: 28142

Searching for "Follow" by Amelie Lens...
Done.
Token count: 345
Adding lyrics to row...
Index: 28143

Searching for "Steam Roller" by Paul Ritch...
Done.
Token count: 4682
Adding lyrics to row...
Index: 28144

Searching for "Bipolar Star - Victor Ruiz Remix" by Olivier Giacomotto...
No results found for: 'Bipolar Star - Victor Ruiz Remix Olivier Giacomotto'
No result: Adding 'no result' to row...
Index: 28145

Searching for "Sad But True" by Boris Brejcha...
Done.
Token count: 148557
Adding lyrics to row...
Index: 28146

Searching for "Yoru" by Rafael Cerato...
Done.
Token count: 3227
Adding lyrics to row...
Index: 28147

Searching for "Moments - Original Mix" by Chris Lawyer...
Done.
Token count: 12235
Adding lyrics to row...
Index: 28148

Searching for "Hypnotic Energy" by Liquid Soul...
Done.
Token count: 43
Addi

Done.
Token count: 206
Adding lyrics to row...
Index: 28207

Searching for "I Like This F’n Song" by Deorro...
Done.
Token count: 5974
Adding lyrics to row...
Index: 28208

Searching for "I See" by Nicky Romero...
Done.
Token count: 103
Adding lyrics to row...
Index: 28209

Searching for "Kick, Bass & Trance" by Andrew Rayel...
Done.
Token count: 2387
Adding lyrics to row...
Index: 28210

Searching for "Here Tonight" by Wildcrow...
Done.
Token count: 321
Adding lyrics to row...
Index: 28211

Searching for "Love Again - KIIDA Remix" by D-wayne...
Done.
Token count: 3613
Adding lyrics to row...
Index: 28212

Searching for "Headshot" by Dave Mak...
Done.
Token count: 623
Adding lyrics to row...
Index: 28213

Searching for "Raaja" by Noiz Van Grane...
Done.
Token count: 488
Adding lyrics to row...
Index: 28214

Searching for "Retumba" by Deorro...
Specified song does not contain lyrics. Rejecting.
No result: Adding 'no result' to row...
Index: 28215

Searching for "Trap 4 Real" by Diego Mi

Done.
Token count: 324
Adding lyrics to row...
Index: 28275

Searching for "Under Control (feat. Hurts)" by Calvin Harris...
Done.
Token count: 179
Adding lyrics to row...
Index: 28276

Searching for "Never Say Goodbye - Radio Edit" by Hardwell...
Done.
Token count: 63
Adding lyrics to row...
Index: 28277

Searching for "Wake Me Up" by Avicii...
Done.
Token count: 263
Adding lyrics to row...
Index: 28278

Searching for "This Is What It Feels Like" by Armin van Buuren...
Done.
Token count: 236
Adding lyrics to row...
Index: 28279

Searching for "Until the End (feat. Joe Gil)" by Dirty South...
Done.
Token count: 213
Adding lyrics to row...
Index: 28280

Searching for "Thunder - Radio Edit" by W&W...
Done.
Token count: 366
Adding lyrics to row...
Index: 28281

Searching for "Get Up (Rattle) - Vocal Extended Version" by Bingo Players...
Done.
Token count: 264
Adding lyrics to row...
Index: 28282

Searching for "Miami 82 - Vocal Radio Mix" by Syn Cole...
No results found for: 'Miami 82 - V

Done.
Token count: 189
Adding lyrics to row...
Index: 28342

Searching for "Closer - Sultan & Ned Shepard Remix" by Tegan and Sara...
No results found for: 'Closer - Sultan & Ned Shepard Remix Tegan and Sara'
No result: Adding 'no result' to row...
Index: 28343

Searching for "Sweet Surrender - Radio Edit" by Starkillers...
Done.
Token count: 174
Adding lyrics to row...
Index: 28344

Searching for "Only For You - Maor Levi Remix" by Mat Zo...
Done.
Token count: 8819
Adding lyrics to row...
Index: 28345

Searching for "Typhoon - Original Mix" by Julian Calor...
No results found for: 'Typhoon - Original Mix Julian Calor'
No result: Adding 'no result' to row...
Index: 28346



In [67]:
df_fin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28347 entries, 0 to 28346
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   track_id                  28347 non-null  object 
 1   track_name                28347 non-null  object 
 2   track_artist              28347 non-null  object 
 3   track_popularity          28347 non-null  int64  
 4   track_album_id            28347 non-null  object 
 5   track_album_name          28347 non-null  object 
 6   track_album_release_date  28347 non-null  object 
 7   playlist_name             28347 non-null  object 
 8   playlist_id               28347 non-null  object 
 9   playlist_genre            28347 non-null  object 
 10  playlist_subgenre         28347 non-null  object 
 11  danceability              28347 non-null  float64
 12  energy                    28347 non-null  float64
 13  key                       28347 non-null  int64  
 14  loudne

In [68]:
# save the dataset with lyrics
df_fin.to_csv('datasets/all_songs_with_more_genius_lyrics.csv', index=False)

In [70]:
# how many are lyrics were not found?
df_fin[df_fin['track_lyrics']=='no result'].shape

(2499, 25)

There are 2499 songs lyrics of which could not be found. 

I clean these lyrics in the next notebook.